In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Configure device
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper-parameter
input_size = 28
sequence_lenth = 28
num_epochs = 2
hidden_size = 128
num_classes = 10
num_layers = 2
batch_size = 100
learning_rate = 0.003

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../data',
                                           transform=transforms.ToTensor(),
                                           train=True,
                                           download=False)

test_dataset = torchvision.datasets.MNIST(root='../data', 
                                          transform=transforms.ToTensor(),
                                          train=False,
                                          download=False)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [3]:
# Define bidirectional RNN
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        
#         self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidrection
        
    def forward(self, x):
        # Initialize initial hidden and cell state
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shage (batch_size, seq_len, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out
    
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimize
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_lenth, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.6319
Epoch [1/2], Step [200/600], Loss: 0.3271
Epoch [1/2], Step [300/600], Loss: 0.2515
Epoch [1/2], Step [400/600], Loss: 0.1493
Epoch [1/2], Step [500/600], Loss: 0.3002
Epoch [1/2], Step [600/600], Loss: 0.2074
Epoch [2/2], Step [100/600], Loss: 0.1635
Epoch [2/2], Step [200/600], Loss: 0.0799
Epoch [2/2], Step [300/600], Loss: 0.2052
Epoch [2/2], Step [400/600], Loss: 0.0567
Epoch [2/2], Step [500/600], Loss: 0.0614
Epoch [2/2], Step [600/600], Loss: 0.0665


In [6]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_lenth, input_size).to(device)
        labels = labels.to(device)
        
        outputs, _ = model(images)
        predicted = torch.max(outputs.data, 1)
        
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
    print('Test Accuracy over 10000 test images: {}%'.format(100 * correct / total))
    
# Save model
torch.save(model.state_dict(), 'model.ckpt')

ValueError: too many values to unpack (expected 2)